# **Practical 5**

# **Text Classification using Naïve Bayes, SVM and Evaluating Classification Models.**

In [1]:
#!pip install xlrd

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.stem import PorterStemmer
porter = PorterStemmer()
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

In [3]:
df=pd.read_excel("Dataset/Data_Train.xlsx")

In [4]:
df.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [5]:
df.isnull().sum()

STORY      0
SECTION    0
dtype: int64

In [6]:
df.STORY=df.STORY.apply(lambda x:re.sub(r"[^A-Za-z]"," ",x))

In [7]:
def clean_data(row):
    tokens=row.split()
    word_list=[]
    for x in tokens:
        x=porter.stem(x.lower())
        if (x not in stop_words) and len(x)>2:
            word_list.append(x.lower())
    return (" ".join(word_list))   

In [8]:
df['story']=df.STORY.apply(lambda x:clean_data(x))

In [9]:
df.head()

,STORY,SECTION,story
0,But the most painful was the huge reversal in ...,3,pain huge revers fee incom unheard among priva...
1,How formidable is the opposition alliance amon...,0,formid opposit allianc among congress jharkhan...
2,Most Asian currencies were trading lower today...,3,asian currenc trade lower today south korean c...
3,If you want to answer any question click on ...,1,want answer ani question click answer click an...
4,In global markets gold prices edged up today ...,3,global market gold price edg today disappoint ...


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df['story'], 
    df['SECTION'], 
    random_state = 1
)

In [11]:
print("size of training dataset:{0}\nsize of testing dataset:{1}".format(len(X_train),len(X_test)))

size of training dataset:5721
size of testing dataset:1907


In [12]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(y_train)
Test_Y = Encoder.fit_transform(y_test)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['story'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [14]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  96.27687467226009


In [15]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  97.48295752490823
